In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,f1_score
from sklearn.ensemble import GradientBoostingRegressor,BaggingClassifier,RandomForestRegressor ,RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import warnings 
warnings.filterwarnings("ignore")
from imblearn.over_sampling import RandomOverSampler, SMOTE, SMOTENC
from imblearn.under_sampling import RandomUnderSampler, NearMiss, TomekLinks, EditedNearestNeighbours
from imblearn.combine import SMOTETomek, SMOTEENN
from scipy.stats import chi2_contingency
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
#---------------
from time import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
from sklearn import (manifold, datasets, decomposition, ensemble, random_projection)
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

from sklearn.preprocessing import LabelEncoder

In [3]:
train = pd.read_csv("Train-Set.csv")
test = pd.read_csv("Test-Set.csv")
train.head(1)


,Unnamed: 0,id,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,Target
0,56963,31963,47,blue-collar,married,basic.9y,no,NaN,no,yes,cellular,apr,fri,583,2,999,1,failure,no


In [4]:
train['t_type'] = 'train'
test['t_type'] = 'test'

df = train.append(test, ignore_index=True)

In [5]:
def unknown(data_frame):
    columns = data_frame.columns

    presence_list = []
    for i in columns:
        if 'unknown' in data_frame[i].unique():
            presence_list.append(i)
    return presence_list

unknowns = unknown(df)
unknowns

['job',
 'marital',
 'education',
 'default',
 'housing',
 'loan',
 'contact',
 'poutcome']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78161 entries, 0 to 78160
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  78161 non-null  int64  
 1   id          78161 non-null  int64  
 2   age         78161 non-null  int64  
 3   job         78161 non-null  object 
 4   marital     78161 non-null  object 
 5   education   78161 non-null  object 
 6   default     78161 non-null  object 
 7   balance     45211 non-null  float64
 8   housing     78161 non-null  object 
 9   loan        78161 non-null  object 
 10  contact     78161 non-null  object 
 11  day         78161 non-null  object 
 12  month       78161 non-null  object 
 13  duration    78161 non-null  int64  
 14  campaign    78161 non-null  int64  
 15  pdays       78161 non-null  int64  
 16  previous    78161 non-null  int64  
 17  poutcome    78161 non-null  object 
 18  Target      54712 non-null  object 
 19  t_type      78161 non-nul

In [7]:
for i in unknowns:
    precentage = len(df[df[i] == 'unknown']) /len(df[i]) * 100
    print(i.upper() , f'\n=========================================\n{str(precentage)}' + '%' '\n\n\n' )

JOB 
0.7075139775591407%



MARITAL 
0.08316167909827152%



EDUCATION 
4.163201596704239%



DEFAULT 
8.879108506800067%



HOUSING 
1.0184107163419098%



LOAN 
1.0184107163419098%



CONTACT 
16.657924028607617%



POUTCOME 
47.285730735277184%





In [8]:
#df.drop(['Unnamed: 0','poutcome','balance','job'],axis=1,inplace=True)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78161 entries, 0 to 78160
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  78161 non-null  int64  
 1   id          78161 non-null  int64  
 2   age         78161 non-null  int64  
 3   job         78161 non-null  object 
 4   marital     78161 non-null  object 
 5   education   78161 non-null  object 
 6   default     78161 non-null  object 
 7   balance     45211 non-null  float64
 8   housing     78161 non-null  object 
 9   loan        78161 non-null  object 
 10  contact     78161 non-null  object 
 11  day         78161 non-null  object 
 12  month       78161 non-null  object 
 13  duration    78161 non-null  int64  
 14  campaign    78161 non-null  int64  
 15  pdays       78161 non-null  int64  
 16  previous    78161 non-null  int64  
 17  poutcome    78161 non-null  object 
 18  Target      54712 non-null  object 
 19  t_type      78161 non-nul

In [10]:
df[df == 'unknown'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78161 entries, 0 to 78160
Data columns (total 20 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  0 non-null      float64
 1   id          0 non-null      float64
 2   age         0 non-null      float64
 3   job         553 non-null    object 
 4   marital     65 non-null     object 
 5   education   3254 non-null   object 
 6   default     6940 non-null   object 
 7   balance     0 non-null      float64
 8   housing     796 non-null    object 
 9   loan        796 non-null    object 
 10  contact     13020 non-null  object 
 11  day         0 non-null      object 
 12  month       0 non-null      object 
 13  duration    0 non-null      float64
 14  campaign    0 non-null      float64
 15  pdays       0 non-null      float64
 16  previous    0 non-null      float64
 17  poutcome    36959 non-null  object 
 18  Target      0 non-null      object 
 19  t_type      0 non-null   

In [11]:
df.loc[(train['job'] == 'unknown') & (df['age'] > 60), 'job'] = 'retired'
df.loc[(train['job'] == 'unknown') & ((df['education'] == 'basic.4y')|(df['education'] == 'basic.6y')|(df['education'] == 'basic.9y')), 'job'] = 'blue-collar'
df.loc[(train['job'] == 'unknown') & ((df['education'] == 'secondary')|(df['education'] == 'primary')), 'job'] = 'blue-collar'
df.loc[(train['job'] == 'unknown') & (df['education'] == 'tertiary'), 'job'] = 'management'
df.loc[(train['job'] == 'unknown') & ((df['education'] == 'high.school')|(df['education'] == 'high.school')), 'job'] = 'admin.'
df.loc[(train['job'] == 'unknown') & ((df['education'] == 'professional.course')|(df['education'] == 'tertiary')), 'job'] = 'technician'
df.loc[(train['job'] == 'unknown') & (df['age'] < 20), 'job'] = 'student'
df.loc[df.job == 'unknown','job'] = pd.Series.mode(df.job)

In [12]:
jobs_salary = dict(df.balance.groupby(df.job).mean())
#jobs_salary.pop('unknown')
for job in jobs_salary:
    df.loc[df.job == job,'balance'] = df.loc[df.job == job,'balance'].fillna(jobs_salary[job])
df.balance.fillna(df.balance.mean(),inplace=True)

In [13]:
df.loc[(df.age < 30) & (df.marital == 'unknown'),'marital'] = 'single'
df.loc[(df.age > 30) & (df.housing == 'yes') & (df.marital == 'unknown') ,'marital'] = 'married'
df.loc[(df.marital == 'unknown'),'marital'] = 'married'

In [14]:
df.loc[df['poutcome'] == 'other', 'poutcome'] = 'unknown'
df.loc[df['poutcome'] == 'nonexistent', 'poutcome'] = 'unknown'

In [15]:
df.loc[(df['education'] == 'unknown') & (df['job'] == 'admin.'), 'education'] = 'university.degree'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'blue-collar'), 'education'] = 'secondary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'technician'), 'education'] = 'secondary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'management'), 'education'] = 'tertiary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'entrepreneur'), 'education'] = 'tertiary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'housemaid'), 'education'] = 'primary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'self-employed'), 'education'] = 'tertiary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'services'), 'education'] = 'secondary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'student'), 'education'] = 'secondary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'unemployed'), 'education'] = 'secondary'
df.loc[(df['education'] == 'unknown') & (df['job'] == 'retired'), 'education'] = 'secondary'
df.loc[df.education == 'unknown','education'] = pd.Series.mode(df.education)

In [16]:
df.loc[(df.balance < 1000) & (df.default == 'unknown'),'default'] = 'no'
df.loc[(df.default == 'unknown'),'default'] = 'yes'

In [17]:
df.loc[(df.default == 'no'),'housing'] = 'yes'
df.loc[(df.default == 'yes'),'housing'] = 'no'

In [18]:
df.loc[(df.default == 'no') & (df.housing =='yes'),'loan'] = 'no'
df.loc[(df.default == 'yes') & (df.housing =='no'),'loan'] = 'yes'

In [19]:
df.contact.replace({'unknown':pd.NA},inplace=True)
df.contact.fillna(method='bfill',inplace=True)

---
---
---

In [20]:
bins = [0, 20, 40, 60, 80, 100]
labels = ['young', 'adult', 'middle-aged', 'senior', 'elderly']

# Create a new categorical column 'age_group' based on the bins
df['age'] = pd.cut(df['age'], bins=bins, labels=labels)

## Education 

In [21]:
df.loc[df.education.isin(['basic.4y','basic.6y','basic.9y']),'education'] = 'custom group'

# day , month

In [22]:
outDays = [ x for x in df.day.unique() if not x.isdigit()]
for i in outDays:
    condition = df.day == i
    temp = df.loc[condition,'day']
    
    df.loc[condition,'day'] = df.loc[condition,'month']
    df.loc[condition,'month'] = temp
# shifting month and day

In [23]:
days = {'1':'mon','2':'tue','3':'wed','4':'thu','5':'fri','6':'sat','7':'sun'}
def conv_day(d):
    if d.isdigit():
        for k in list(days.keys())[::-1]:
            if int(d)%int(k) == 0:
                return days[k]
    return 'mon'
df['day'] = df.day.apply(lambda d:conv_day(d))

In [24]:
df.month.value_counts()

may    24777
jul    12658
aug    11195
jun     9588
nov     7236
apr     5017
feb     2649
jan     1403
oct     1325
sep     1043
mar      913
dec      357
Name: month, dtype: int64

# we should drop duration

In [25]:
def campaign(x):
    if x > 10:
        return 'high'
    elif x > 5:
        return 'mid'
    return 'low'
df['campaign'] = df.campaign.apply(lambda d: campaign(d))

In [26]:
def rePdays(x):
    if x in [-1,999]:
        return 'from too long'
    elif x > 200:
        return 'from short period'
    return 'from short time'
df['pdays'] = df.pdays.apply(lambda d:rePdays(d))

In [27]:
def rePrevious(p):
    if p > 18:
        return 'high'
    elif p > 10:
        return 'mid-high'
    elif p >= 1:
        return 'low'
    return 'No'
df['previous'] = df.previous.apply(lambda d:rePrevious(d))

In [28]:
df.drop(labels = ['Unnamed: 0','duration','poutcome'],axis=1,inplace=True)

In [ ]:
df.to_csv('Full data.csv', index=False)

# -----------------------------------------------------------------------------------

In [29]:
from sklearn.preprocessing import LabelEncoder
for col in df:
    if col not in ['balance','id','t_type','Target']:
        L =LabelEncoder()
        df[col] = L.fit_transform(df[col])
        


In [30]:
train = df.loc[df.t_type == 'train',].iloc[:,1:-1]

In [31]:
'''# Downsampling function
# Downsample the input 'DataFrame' by balancing the class distribution.
def returnbalanceData(data,target,min_class):
    
    classes = dict(data[target].value_counts())
    data.reindex()
    index = np.array([])
    for clas in classes:
        class_index = data[data[target] == clas].index
        sample=list(np.random.choice(class_index,classes[min_class],replace=False))
        
        index = np.concatenate((sample,index))
        
    return data.iloc[index,:]
train = returnbalanceData(train,'Target','yes')'''

In [59]:
X = train.drop('Target',axis=1).values
y = train['Target']
y = y.map({'yes': 1, 'no': 0})
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [62]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)

In [63]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.59      0.76      0.66      1239
           1       0.67      0.48      0.56      1273

    accuracy                           0.62      2512
   macro avg       0.63      0.62      0.61      2512
weighted avg       0.63      0.62      0.61      2512



In [64]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion="gini", random_state=42,max_depth=3, min_samples_leaf=5)
dt.fit(X_train,y_train)
y_pred = dt.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.60      0.89      0.71      1239
           1       0.80      0.41      0.54      1273

    accuracy                           0.65      2512
   macro avg       0.70      0.65      0.63      2512
weighted avg       0.70      0.65      0.63      2512



In [65]:
test = df.loc[df.t_type == 'test',].iloc[:,:-2]
pred = svm.predict(test.iloc[:,1:])


array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [66]:
submission =pd.read_csv('Sample_Submition.csv')
# Create a submisison dataframe and append the relevant columns
submission = pd.DataFrame()
submission['id'] = test['id']
submission['Target'] =pred # our model predictions on the test dataset
submission.to_csv('Asubmission.csv', index=False)

In [ ]:
from sklearn.svm import SVC
grid = {
   'C':[0.01,0.1,1,10],
   'kernel' : ["linear","poly","rbf","sigmoid"],
   'degree' : [1,2,3,5,7],
   'gamma' : [0.01,1,0.001]
}
svm  = SVC ()
# GridSearchCV is a useful tool to fine tune the parameters of your model.
# The “best” parameters that GridSearchCV identifies are technically the best that could be produced
svm_cv = GridSearchCV(svm, grid, cv = 5)
svm_cv.fit(X_train,y_train)
print("Best Parameters:",svm_cv.best_params_)
print("Train Score:",svm_cv.best_score_)
print("Test Score:",svm_cv.score(X_test,y_test))

In [76]:
from sklearn.svm import SVC
svm  = SVC (C=0.01,kernel='rbf',degree = 2,gamma=0.01)
svm.fit(X_train,y_train)
print("Test Score:",f1_score(y_test,svm.predict(X_test)))

Test Score: 0.6486817903126917


In [79]:
pred = svm.predict(test.iloc[:,1:])


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [78]:
submission =pd.read_csv('Sample_Submition.csv')
# Create a submisison dataframe and append the relevant columns
submission = pd.DataFrame()
submission['id'] = test['id']
submission['Target'] =pred # our model predictions on the test dataset
submission.to_csv('Asubmission.csv', index=False)

In [83]:
submission['Target'] = submission.Target.apply(lambda X: 0 if X==1 else 1)

In [84]:
submission.to_csv('Asubmission.csv', index=False)